In [2]:
from searchtweets import ResultStream, gen_rule_payload, load_credentials, collect_results
import csv
import pandas
import pandas as pd
import numpy as np
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob 
%matplotlib inline
import re
import time
import math

In [3]:
#setting up credentials

In [4]:
enterprise_search_args = load_credentials("~/.twitter_keys.yaml",
                                          yaml_key="search_tweets_api",
                                          env_overwrite=False)

Grabbing bearer token from OAUTH


In [5]:
#loading data

In [6]:
data = pandas.read_csv("~/Documents/nfl sentiment/final_data/final_subset.csv")

In [7]:
#looking at data

In [8]:
data.head()

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
0,Russell Wilson,@DangeRussWilson',Seattle Seahawks,B,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Russell Wilson OR @DangeRussWilson'
1,Doug Baldwin,@DougBaldwinJr',Seattle Seahawks,B,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Doug Baldwin OR @DougBaldwinJr'
2,Jimmy Graham,@TheJimmyGraham',Seattle Seahawks,W,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Jimmy Graham OR @TheJimmyGraham'
3,Cam Newton,@CameronNewton',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Cam Newton OR @CameronNewton'
4,Devin Funchess,@D_FUNCH',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Devin Funchess OR @D_FUNCH'


In [9]:
#creating function to filter by team

In [10]:
def get_data(teams):
    data_1 = data[data.Team.isin(teams)]
    
    return data_1

In [11]:
#creating function to get tweets for those teams

In [31]:
def get_tweets(data, start, end):
    
    #getting the twitter handles
    twitter = data.full_twitter.tolist()

    newtwitter = []

    for i in range(len(twitter)):
        a = twitter[i].replace("'", "")
    
        newtwitter.append(a)
    
    print(newtwitter)
    
    
    
    #getting start date
    
    start = data[start]
    
    start = start.tolist()

    start = start[1].replace("'", "")
    
    print(start)
    
    #getting end date
    
    end = data[end]
    
    end = end.tolist()

    end = end[1].replace("'", "")
    
    print(end)
    
    
    
    #running loop for tweets
    
    all_tweets = []

    some_tweets = []

    for i in range(4):

        for handle in newtwitter:

            rule = gen_rule_payload(handle + "-is:retweet",
                                    from_date = start,
                                    to_date = end,
                                    results_per_call = 500)

            tweets2 = collect_results(rule, max_results=500, result_stream_args=enterprise_search_args)

            [print(tweet.all_text) for tweet in tweets2[0:10]];
        
            some_tweets.extend(tweets2)
        
            time.sleep(1)
        
        all_tweets.extend(some_tweets)
        
        
        
    #creating df    
    
    
    
    # We create a pandas dataframe as follows:
    data_tweets = pd.DataFrame(data=[tweet.text for tweet in all_tweets], columns=['Tweets'])
    
    #adding more columns
    data_tweets['len']  = np.array([len(tweet.text) for tweet in all_tweets])
    data_tweets['ID']   = np.array([tweet.id for tweet in all_tweets])
    data_tweets['Date'] = np.array([tweet.created_at_datetime for tweet in all_tweets])
    data_tweets['Likes']  = np.array([tweet.favorite_count for tweet in all_tweets])
    data_tweets['RTs']    = np.array([tweet.retweet_count for tweet in all_tweets])
    data_tweets['Quoted'] = np.array([tweet.quoted_tweet for tweet in all_tweets])
    data_tweets['Q_or_RT'] = np.array([tweet.quote_or_rt_text for tweet in all_tweets])
    data_tweets['User_ent_text'] = np.array([tweet.user_entered_text for tweet in all_tweets])
    data_tweets['retweeted_tweet'] = np.array([tweet.retweeted_tweet for tweet in all_tweets])
    data_tweets['user_mentions'] = np.array([tweet.user_mentions for tweet in all_tweets])
    data_tweets['profile_location'] = np.array([tweet.profile_location for tweet in all_tweets])
    data_tweets['in_reply_to_screen_name'] = np.array([tweet.in_reply_to_screen_name for tweet in all_tweets])
    data_tweets['created_at_string'] = np.array([tweet.created_at_string for tweet in all_tweets])
    data_tweets['tweet_type'] = np.array([tweet.tweet_type for tweet in all_tweets])
    data_tweets['all_text'] = np.array([tweet.all_text for tweet in all_tweets])
    
    return data_tweets

In [ ]:
####T1-1

In [ ]:
#getting data

In [13]:
teams = ["Kansas City Chiefs", "Tampa Bay Buccaneers", "Baltimore Ravens", "Philadelphia Eagles", "Carolina Panthers"]
data_t1_1 = get_data(teams)

In [14]:
data_t1_1

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
3,Cam Newton,@CameronNewton',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Cam Newton OR @CameronNewton'
4,Devin Funchess,@D_FUNCH',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Devin Funchess OR @D_FUNCH'
5,Christian McCaffrey,@run__cmc',Carolina Panthers,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Christian McCaffrey OR @run__cmc'
9,Jameis Winston,@Jaboowins',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Jameis Winston OR @Jaboowins'
10,Mike Evans,@MikeEvans13_',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Mike Evans OR @MikeEvans13_'
11,Adam Humphries,@adamhumphries13',Tampa Bay Buccaneers,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Adam Humphries OR @adamhumphries13'
12,Carson Wentz,@cj_wentz',Philadelphia Eagles,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Carson Wentz OR @cj_wentz'
13,Zach Ertz,@ZERTZ_86',Philadelphia Eagles,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Zach Ertz OR @ZERTZ_86'
14,Alshon Jeffrey,@TheWorldof_AJ',Philadelphia Eagles,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Alshon Jeffrey OR @TheWorldof_AJ'
15,Alex Smith,NaN,Kansas City Chiefs,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 21:25',2017-09-25 01:25',2017-10-03 01:30',2017-10-03 05:30',2017-10-09 01:30',2017-10-09 05:30',2017-10-15 21:25',2017-10-16 01:25',Alex Smith


In [ ]:
#getting tweets

In [ ]:
data_t1_1_tweets = get_tweets(data_t1_1, start = 'T1_start', end = 'T1_end')

In [ ]:
#saving at csv

In [ ]:
data_t1_1_tweets.to_csv("~/Documents/nfl sentiment/final_data/tweets_t1_1.csv", index = False)

In [ ]:
####T1-2

In [ ]:
#getting data

In [26]:
teams = ["Dallas Cowboys"]
data_t1_2 = get_data(teams)

In [16]:
data_t1_2

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [ ]:
#getting tweets

In [ ]:
data_t1_2_tweets = get_tweets(data_t1_2, start = 'T1_start', end = 'T1_end')

In [ ]:
#saving at csv

In [ ]:
data_t1_2_tweets.to_csv("~/Documents/nfl sentiment/final_data/tweets_t1_2.csv", index = False)

In [ ]:
####T1-3

In [ ]:
#getting data

In [17]:
teams = ["Seattle Seahawks"]
data_t1_3 = get_data(teams)

In [18]:
data_t1_3

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
0,Russell Wilson,@DangeRussWilson',Seattle Seahawks,B,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Russell Wilson OR @DangeRussWilson'
1,Doug Baldwin,@DougBaldwinJr',Seattle Seahawks,B,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Doug Baldwin OR @DougBaldwinJr'
2,Jimmy Graham,@TheJimmyGraham',Seattle Seahawks,W,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Jimmy Graham OR @TheJimmyGraham'


In [ ]:
#getting tweets

In [ ]:
data_t1_3_tweets = get_tweets(data_t1_3, start = 'T1_start', end = 'T1_end')

In [ ]:
#saving at csv

In [ ]:
data_t1_3_tweets.to_csv("~/Documents/nfl sentiment/final_data/tweets_t1_3.csv", index = False)

In [ ]:
####T1-4

In [ ]:
#getting data

In [21]:
teams = ["Cincinnati Bengals"]
data_t1_4 = get_data(teams)

In [22]:
data_t1_4

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
18,Andy Dalton,@andydalton14',Cincinnati Bengals,W,2017-09-10 18:00',2017-09-10 22:00',2017-09-15 01:25',2017-09-15 05:25',2017-09-24 21:25',2017-09-25 01:25',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',Andy Dalton OR @andydalton14'
19,Tyler Kroft,@Kroft86',Cincinnati Bengals,W,2017-09-10 18:00',2017-09-10 22:00',2017-09-15 01:25',2017-09-15 05:25',2017-09-24 21:25',2017-09-25 01:25',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',Tyler Kroft OR @Kroft86'
20,A.J. Green,@ajgreen_18',Cincinnati Bengals,B,2017-09-10 18:00',2017-09-10 22:00',2017-09-15 01:25',2017-09-15 05:25',2017-09-24 21:25',2017-09-25 01:25',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',A.J. Green OR @ajgreen_18'


In [ ]:
#getting tweets

In [ ]:
data_t1_4_tweets = get_tweets(data_t1_4, start = 'T1_start', end = 'T1_end')

In [ ]:
#saving at csv

In [ ]:
data_t1_4_tweets.to_csv("~/Documents/nfl sentiment/final_data/tweets_t1_4.csv", index = False)

In [ ]:
#appending for T1

In [ ]:
T1 = pd.concat([data_t1_1_tweets, data_t1_2_tweets, data_t1_3_tweets, data_t1_4_tweets])

In [ ]:
T1.to_csv("~/Documents/nfl sentiment/final_data/tweets_t1_all.csv", index = False)

In [ ]:
####T2-1

In [ ]:
#getting data

In [13]:
teams = ["Kansas City Chiefs"]
data_t2_1 = get_data(teams)

In [14]:
data_t2_1

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
3,Cam Newton,@CameronNewton',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Cam Newton OR @CameronNewton'
4,Devin Funchess,@D_FUNCH',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Devin Funchess OR @D_FUNCH'
5,Christian McCaffrey,@run__cmc',Carolina Panthers,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Christian McCaffrey OR @run__cmc'
9,Jameis Winston,@Jaboowins',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Jameis Winston OR @Jaboowins'
10,Mike Evans,@MikeEvans13_',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Mike Evans OR @MikeEvans13_'
11,Adam Humphries,@adamhumphries13',Tampa Bay Buccaneers,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Adam Humphries OR @adamhumphries13'
12,Carson Wentz,@cj_wentz',Philadelphia Eagles,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Carson Wentz OR @cj_wentz'
13,Zach Ertz,@ZERTZ_86',Philadelphia Eagles,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Zach Ertz OR @ZERTZ_86'
14,Alshon Jeffrey,@TheWorldof_AJ',Philadelphia Eagles,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Alshon Jeffrey OR @TheWorldof_AJ'
15,Alex Smith,NaN,Kansas City Chiefs,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 21:25',2017-09-25 01:25',2017-10-03 01:30',2017-10-03 05:30',2017-10-09 01:30',2017-10-09 05:30',2017-10-15 21:25',2017-10-16 01:25',Alex Smith


In [ ]:
#getting tweets

In [ ]:
data_t2_1_tweets = get_tweets(data_t2_1, start = 'T2_start', end = 'T2_end')

In [ ]:
#saving at csv

In [ ]:
data_t2_1_tweets.to_csv("~/Documents/nfl sentiment/final_data/tweets_t2_1.csv", index = False)

In [ ]:
####T2-2

In [ ]:
#getting data

In [34]:
teams = ["Tampa Bay Buccaneers", "Philadelphia Eagles", "Carolina Panthers"]
data_t2_2 = get_data(teams)

In [35]:
data_t2_2

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
3,Cam Newton,@CameronNewton',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Cam Newton OR @CameronNewton'
4,Devin Funchess,@D_FUNCH',Carolina Panthers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Devin Funchess OR @D_FUNCH'
5,Christian McCaffrey,@run__cmc',Carolina Panthers,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Christian McCaffrey OR @run__cmc'
9,Jameis Winston,@Jaboowins',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Jameis Winston OR @Jaboowins'
10,Mike Evans,@MikeEvans13_',Tampa Bay Buccaneers,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Mike Evans OR @MikeEvans13_'
11,Adam Humphries,@adamhumphries13',Tampa Bay Buccaneers,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-06 01:25',2017-10-06 05:25',2017-10-15 21:05',2017-10-16 01:05',Adam Humphries OR @adamhumphries13'
12,Carson Wentz,@cj_wentz',Philadelphia Eagles,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Carson Wentz OR @cj_wentz'
13,Zach Ertz,@ZERTZ_86',Philadelphia Eagles,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Zach Ertz OR @ZERTZ_86'
14,Alshon Jeffrey,@TheWorldof_AJ',Philadelphia Eagles,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 18:00',2017-09-24 22:00',2017-10-01 21:05',2017-10-02 01:05',2017-10-08 18:00',2017-10-08 22:00',2017-10-13 01:25',2017-10-13 05:25',Alshon Jeffrey OR @TheWorldof_AJ'


In [ ]:
#getting tweets

In [ ]:
data_t2_2_tweets = get_tweets(data_t2_2, start = 'T2_start', end = 'T2_end')

In [ ]:
#saving at csv

In [ ]:
data_t2_2_tweets.to_csv("~/Documents/nfl sentiment/final_data/tweets_t2_2.csv", index = False)

In [ ]:
####T2-3

In [ ]:
#getting data

In [36]:
teams = ["Baltimore Ravens"]
data_t2_3 = get_data(teams)

In [37]:
data_t2_3

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
21,Joe Flacco,@JoeFlacco',Baltimore Ravens,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 14:30',2017-09-24 18:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:05',2017-10-09 01:05',2017-10-15 18:00',2017-10-15 22:00',Joe Flacco OR @JoeFlacco'
22,Nick Boyle,@nickboyle86',Baltimore Ravens,W,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 14:30',2017-09-24 18:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:05',2017-10-09 01:05',2017-10-15 18:00',2017-10-15 22:00',Nick Boyle OR @nickboyle86'
23,Mike Wallace,@Wallace17_daKid',Baltimore Ravens,B,2017-09-17 18:00',2017-09-17 22:00',2017-09-24 14:30',2017-09-24 18:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:05',2017-10-09 01:05',2017-10-15 18:00',2017-10-15 22:00',Mike Wallace OR @Wallace17_daKid'


In [ ]:
#getting tweets

In [ ]:
data_t2_3_tweets = get_tweets(data_t2_3, start = 'T2_start', end = 'T2_end')

In [ ]:
#saving at csv

In [ ]:
data_t2_3_tweets.to_csv("~/Documents/nfl sentiment/final_data/tweets_t2_3.csv", index = False)

In [ ]:
####T2-4

In [ ]:
#getting data

In [38]:
teams = ["Dallas Cowboys", "Seattle Seahawks"]
data_t2_4 = get_data(teams)

In [39]:
data_t2_4

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
0,Russell Wilson,@DangeRussWilson',Seattle Seahawks,B,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Russell Wilson OR @DangeRussWilson'
1,Doug Baldwin,@DougBaldwinJr',Seattle Seahawks,B,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Doug Baldwin OR @DougBaldwinJr'
2,Jimmy Graham,@TheJimmyGraham',Seattle Seahawks,W,2017-09-10 21:25',2017-09-11 01:25',2017-09-17 21:25',2017-09-18 01:25',2017-09-24 21:05',2017-09-25 01:05',2017-10-02 01:30',2017-10-02 05:30',2017-10-08 21:05',2017-10-09 01:05',Jimmy Graham OR @TheJimmyGraham'
6,Dak Prescott,@dak',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dak Prescott OR @dak'
7,Dez Bryant,@DezBryant',Dallas Cowboys,B,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Dez Bryant OR @DezBryant'
8,Jason Witten,@JasonWitten',Dallas Cowboys,W,2017-09-11 01:30',2017-09-11 05:30',2017-09-17 21:25',2017-09-18 01:25',2017-09-26 01:30',2017-09-26 05:30',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 21:25',2017-10-09 01:25',Jason Witten OR @JasonWitten'


In [ ]:
#getting tweets

In [ ]:
data_t2_4_tweets = get_tweets(data_t2_4, start = 'T2_start', end = 'T2_end')

In [ ]:
#saving at csv

In [ ]:
data_t2_4_tweets.to_csv("~/Documents/nfl sentiment/final_data/tweets_t2_4.csv", index = False)

In [ ]:
####T2-5

In [ ]:
#getting data

In [40]:
teams = ["Cincinnati Bengals"]
data_t2_5 = get_data(teams)

In [41]:
data_t2_5

,Name,Twitter_handle,Team,Race,T1_start,T1_end,T2_start,T2_end,T3_start,T3_end,T4_start,T4_end,T5_start,T5_end,full_twitter
18,Andy Dalton,@andydalton14',Cincinnati Bengals,W,2017-09-10 18:00',2017-09-10 22:00',2017-09-15 01:25',2017-09-15 05:25',2017-09-24 21:25',2017-09-25 01:25',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',Andy Dalton OR @andydalton14'
19,Tyler Kroft,@Kroft86',Cincinnati Bengals,W,2017-09-10 18:00',2017-09-10 22:00',2017-09-15 01:25',2017-09-15 05:25',2017-09-24 21:25',2017-09-25 01:25',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',Tyler Kroft OR @Kroft86'
20,A.J. Green,@ajgreen_18',Cincinnati Bengals,B,2017-09-10 18:00',2017-09-10 22:00',2017-09-15 01:25',2017-09-15 05:25',2017-09-24 21:25',2017-09-25 01:25',2017-10-01 18:00',2017-10-01 22:00',2017-10-08 18:00',2017-10-08 22:00',A.J. Green OR @ajgreen_18'


In [ ]:
#getting tweets

In [ ]:
data_t2_5_tweets = get_tweets(data_t2_5, start = 'T2_start', end = 'T2_end')

In [ ]:
#saving at csv

In [ ]:
data_t2_5_tweets.to_csv("~/Documents/nfl sentiment/final_data/tweets_t2_5.csv", index = False)

In [ ]:
#appending for T1

In [ ]:
T2 = pd.concat([data_t2_1_tweets, data_t2_2_tweets, data_t2_3_tweets, data_t2_4_tweets, data_t2_5_tweets])

In [ ]:
T2.to_csv("~/Documents/nfl sentiment/final_data/tweets_t2_all.csv", index = False)